# Plots for Thesis Paper

In [ ]:
import util
import math
import numpy as np

from blurs import *

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

## Plots of PSF Functions

In [ ]:
psf_linear = linear((32, 32), *[7, math.pi/4])
psf_gaussian = gaussian((32, 32), *[3, 1.5])
psf_radial = radial((32, 32), 9)

from plotly import subplots
from scipy.fft import ifft2, fftshift

psf_linear = util.normalize(np.real(fftshift(ifft2(psf_linear))))
psf_gaussian = util.normalize(np.real(fftshift(ifft2(psf_gaussian))))
psf_radial = util.normalize(np.real(fftshift(ifft2(psf_radial))))
psfs = [psf_linear, psf_radial, psf_gaussian]

fig = subplots.make_subplots(rows=1, cols=3)
i = 1
for p in psfs:
    fig.add_trace(px.imshow(p).data[0], row=1, col=i)
    i += 1

layout = px.imshow(psf_linear, aspect='equal', color_continuous_scale='gray').layout
fig.layout.coloraxis = layout.coloraxis
fig.update_layout(width=1000, height=300, margin=dict(l=10, r=10, b=10, t=10))
fig.show()

# fig = px.imshow(psf_linear, aspect='equal', color_continuous_scale='gray')
# fig.update_layout(width=400, height=400, margin=dict(l=10, r=10, b=10, t=10))
# fig.show()

# fig = px.imshow(psf_gaussian, aspect='equal', color_continuous_scale='gray')
# fig.update_layout(width=400, height=400, margin=dict(l=10, r=10, b=10, t=10))
# fig.show()

# fig = px.imshow(psf_radial, aspect='equal', color_continuous_scale='gray')
# fig.update_layout(width=400, height=400, margin=dict(l=10, r=10, b=10, t=10))
# fig.show()

# Shapes Blur

In [ ]:
image = util.load_img('../samples/shapes.png')

psf_params = [2, 2]
psf_fn = gaussian
psf = psf_fn(image.shape, *psf_params)

from scipy.fft import ifft2, fftshift
psf_img = np.real(fftshift(ifft2(psf)))
psf_gaussian = util.normalize(np.real(fftshift(ifft2(psf))))

image_blurred = blur(image, psf)

# optional: add noise
noise_factor = 0
image_blurred = noise(image_blurred, noise_factor)

images = [util.normalize(image), image_blurred]

fig = subplots.make_subplots(rows=1, cols=2)
i = 1
for p in images:
    fig.add_trace(px.imshow(p).data[0], row=1, col=i)
    i += 1

layout = px.imshow(psf_gaussian, aspect='equal', color_continuous_scale='gray').layout
fig.layout.coloraxis = layout.coloraxis
fig.update_layout(width=700, height=300, margin=dict(l=10, r=10, b=10, t=10))
fig.show()

# Shapes Deblur

In [ ]:
image = util.load_img('../samples/shapes.png')

psf_params = [1, 1]
psf_fn = gaussian
psf = psf_fn(image.shape, *psf_params)

from scipy.fft import ifft2, fftshift
psf_img = np.real(fftshift(ifft2(psf)))
psf_gaussian = util.normalize(np.real(fftshift(ifft2(psf))))

image_blurred = blur(image, psf)

# optional: add noise
noise_factor = 0.005
image_blurred = noise(image_blurred, noise_factor)

def deblur_tik(alpha, S, blurred_img):
    # S is fft2(circshift(psf))

    # phi = abs(S).^2 ./ (abs(s).^2 + alpha^2)
    # perform adjustment to values which could result in division by zero
    denominator = np.absolute(np.square(S)) + np.square(alpha)
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    phi = np.divide(np.absolute(np.square(S)), denominator)

    # filter out S values according to phi 
    S_filt = np.zeros(phi.shape)
    idx = S != 0
    S_filt[idx] = np.divide(phi[idx], S[idx])

    # deblur using filtered S
    X_filt = ifft2(np.multiply(fft2(blurred_img), S_filt)).real

    return X_filt

image_deblurred = deblur_tik(0, psf, image_blurred)

images = [image_blurred, util.normalize(image_deblurred)]

fig = subplots.make_subplots(rows=1, cols=2)
i = 1
for p in images:
    fig.add_trace(px.imshow(p).data[0], row=1, col=i)
    i += 1

layout = px.imshow(psf_gaussian, aspect='equal', color_continuous_scale='gray').layout
fig.layout.coloraxis = layout.coloraxis
fig.update_layout(width=700, height=300, margin=dict(l=10, r=10, b=10, t=10))
fig.show()